In [1]:
import os

In [2]:
import lightning as L
from data import FakeNewsDataModule, ContrastiveFakeNewsDataModule
from model import NLPModel, SupConModel
import torch

import pandas as pd

In [4]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint

logger = TensorBoardLogger("fakenews_detection", name="RoBERTa Normal")

# log model only if `val_accuracy` increases
checkpoint_callback = ModelCheckpoint(
    monitor="val_accuracy", mode="max", filename='best-checkpoint',  # Name of the checkpoint files
    save_top_k=1,  # Only keep the top 1 model
    verbose=True  # Print when a new checkpoint is saved
)

# trainer = L.Trainer(logger=logger, callbacks=[checkpoint_callback], max_epochs=2)
trainer = L.Trainer(logger=logger, max_epochs=1)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
fakenews_datamodule = ContrastiveFakeNewsDataModule("xlm_fakenews", num_worker=1)

xlm_roberta = SupConModel()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [13]:
trainer.fit(model=xlm_roberta, datamodule=fakenews_datamodule)


  | Name             | Type            | Params
-----------------------------------------------------
0 | model            | XLMRobertaModel | 278 M 
1 | projection_layer | LazyLinear      | 0     
2 | loss             | SupConLoss      | 0     
-----------------------------------------------------
278 M     Trainable params
0         Non-trainable params
278 M     Total params
1,112.175 Total estimated model params size (MB)


/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py:222: UserWarning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1702400235349/work/aten/src/ATen/ParallelNative.cpp:230.)
  torch.set_num_threads(1)


RuntimeError: DataLoader worker (pid(s) 35429) exited unexpectedly

In [ ]:
# from pathlib import Path

# xlm_roberta = NLPModel.load_from_checkpoint(Path("fakenews-detection") / "model.ckpt")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# trainer.test(model=xlm_roberta, datamodule=fakenews_datamodule)